In [ ]:
from numba import njit, prange
import numpy as np
import scipy as sp
import timeit
import os, sys
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
from generate_graph import visualise_graph
import itertools
from read_graph import read_coin_observations, recreate_graph
from scipy.optimize import curve_fit
from scipy import stats
from dynamic import list_of_edges, modify_edge_weight
from simulation_numba import *
from simulation import *

In [ ]:
#value settings:
num_runs = 1000
num_iter = 10000
num_node = 3
fraction = np.arange(0.01, 0.15, 0.02)
simple_edge = True
mod_prob = True
include_zero = False

edge_list = list_of_edges(num_node)

print(edge_list[0])

try:
    overall_output = open('N{}_test_results/overall_output.dat'.format(num_node), 'a') #File contains (in columns) F, total number of runs, avg asymptotic time, median asymptotic time, number of eliminated runs, percentage eliminated
except:
    os.mkdir('N3_test_results')
    overall_output = open('N{}_test_results/overall_output.dat'.format(num_node), 'a') #File contains (in columns) F, total number of runs, avg asymptotic time, median asymptotic time, number of eliminated runs, percentage eliminated


In [ ]:
for frac in fraction:
    runs = 0

    file = open(f'N{num_node}_test_results/F{frac:.4f}.dat', 'a') #File contains (in columns) for probability F: run number, asymptotic time, modification time step, state of N3 network after modification

    print("Running simulation for dynamic probability {:.4f}".format(frac))
    asymp_time_list = []

    while runs < num_runs:

        g = nx.complete_graph(num_node)

        g = randomise_edge_weight(g, edge_weight=(-1, 1), simple_edge_weight=True, include_zero=include_zero)
        
        adj = get_adj_from_nx(g)
        
        beliefs = generate_beliefs(adj_size=num_node)

        asymp_t, _, _, _, modified_edges, modification_timestep = do_simulation_dynamic(adj, edge_list, beliefs, frac, true_bias=0.6, threshold=0.01, time_threshold=100, num_iter=10000, learning_rate=0.25, edge_weight=(-1, 1), simple_edge_weight=simple_edge, mod_probability=mod_prob, include_zero=include_zero)

        asymp_time_list.append(asymp_t)
        
        modified_edges = [list(edge) for edge in modified_edges]

        file.write(f"{runs}\t{asymp_t}\t{modification_timestep}\t{list(modified_edges)}\n")

        runs += 1

    avg_asymp_time, med_asymp_time, tot_runs, runs_eliminated, percentage_eliminated = average_t_asymp(asymp_time_list)

    overall_output.write(f"{frac:.4f}\t{tot_runs}\t{avg_asymp_time}\t{med_asymp_time}\t{runs_eliminated}\t{percentage_eliminated}\n")